In [ ]:
import pandas as pd
import numpy as np

df = pd.read_csv("202311-bluebikes-tripdata 2.csv")  # Data File  - trip history

In [ ]:
# Converted to date and time formart

In [ ]:
df["started_at"] = pd.to_datetime(df["started_at"])
df["ended_at"] = pd.to_datetime(df["ended_at"])

In [ ]:
unique_station_names = df["start_station_name"].unique()

In [ ]:
# Retrieve corresponding station IDs for the unique stations
unique_stations_with_ids = df[df["start_station_name"].isin(unique_station_names)]

In [ ]:
unique_stations_with_ids.head()

In [ ]:
# Optionally, if we want just the station names and IDs:
station_name_id_mapping = unique_stations_with_ids[
    ["start_station_name", "start_station_id", "start_lat", "start_lng"]
]

In [ ]:
station_data = pd.read_csv("current_bluebikes_stations (1).csv")

In [ ]:
weather_station_data = pd.read_csv(
    "Weather_StationData1.csv"
)  # second external File to upload

In [ ]:
import pandas as pd
import numpy as np
from math import radians, sin, cos, sqrt, atan2


# Haversine function to calculate the distance between two lat-long pairs
def haversine(lat1, lon1, lat2, lon2):
    # Convert lat/long from degrees to radians
    lat1, lon1, lat2, lon2 = map(radians, [lat1, lon1, lat2, lon2])

    # Haversine formula
    dlat = lat2 - lat1
    dlon = lon2 - lon1
    a = sin(dlat / 2) ** 2 + cos(lat1) * cos(lat2) * sin(dlon / 2) ** 2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))

    # Radius of Earth in kilometers (mean radius)
    R = 6371.0
    return R * c

In [ ]:
# Create a new column in station_data for the closest weather station
station_data["Closest Weather Station"] = ""
station_data["Weather Station ID"] = ""

# Loop through each station
for i, station in station_data.iterrows():
    station_lat = station["Lat"]
    station_lon = station["Long"]

    # Compute distances to all weather stations
    distances = []
    for j, weather_station in weather_station_data.iterrows():
        weather_lat = weather_station["Approx. Latitude"]
        weather_lon = weather_station["Approx. Longitude"]
        distance = haversine(station_lat, station_lon, weather_lat, weather_lon)
        distances.append(distance)

    # Find the index of the minimum distance
    min_distance_index = np.argmin(distances)

    # Assign the closest weather station
    closest_weather_station = weather_station_data.iloc[min_distance_index]
    station_data.at[i, "Closest Weather Station"] = closest_weather_station[
        "Weather Station Name"
    ]
    station_data.at[i, "Weather Station ID"] = closest_weather_station["Identifier"]

In [ ]:
# Convert both columns to lowercase and strip leading/trailing spaces
df["start_station_name"] = df["start_station_name"].str.lower().str.strip()
station_data["NAME"] = station_data["NAME"].str.lower().str.strip()

# Now perform the merge to get the Closest Weather Station and Weather Station ID for the starting station
df_merged = pd.merge(
    df,
    station_data[
        ["Station ID", "Closest Weather Station", "Weather Station ID", "NAME"]
    ],
    left_on="start_station_name",
    right_on="NAME",
    how="left",
)

# Dropping the redundant 'Station ID' column after the merge
df_merged.drop(columns=["Station ID"], inplace=True)

# df_merged will contain the 'Closest Weather Station' and 'Weather Station ID' for the start station

### Web scrapping

In [ ]:
#!pip install requests

In [ ]:
import time
import pandas as pd
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup
import requests

url = "https://www.wunderground.com/history/daily/KBOS/date/weather/us/ma/boston/42.34,-71.09"

page = requests.get(url)

soup = BeautifulSoup(page.text, "html")

In [ ]:
import time
import pandas as pd
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC


# Set up the WebDriver with Chrome options
def set_up_driver():
    options = webdriver.ChromeOptions()
    options.add_argument("--headless")  # Ensures Chrome runs without a UI
    options.add_argument("--disable-gpu")  # Disable GPU (helps in some headless cases)
    options.add_argument("--no-sandbox")  # Bypass OS security model
    options.add_argument(
        "--disable-dev-shm-usage"
    )  # Overcome limited resource problems
    options.add_argument("--remote-debugging-port=9222")  # Open port for debugging

    driver = webdriver.Chrome(options=options)
    return driver


# Function to scrape weather data for a given date
def scrape_wunderground_data(date):
    driver = set_up_driver()

    # URL format based on the provided link
    url = (
        f"https://www.wunderground.com/history/daily/us/ma/east-boston/KBOS/date/{date}"
    )
    driver.get(url)

    # Wait for the page to load completely
    try:
        # Wait until the table with aria-labelledby="History observation" is present
        WebDriverWait(driver, 20).until(
            EC.presence_of_element_located(
                (By.CSS_SELECTOR, 'table[aria-labelledby="History observation"]')
            )
        )
    except Exception as e:
        print(f"Error: Unable to load table for {date}. {str(e)}")
        driver.quit()
        return None

    # Extract the page source after the table has loaded
    soup = BeautifulSoup(driver.page_source, "html.parser")

    # Now, try to find the table in the loaded HTML
    table = soup.find("table", {"aria-labelledby": "History observation"})

    if not table:
        print(f"No weather data table found for {date}")
        driver.quit()
        return None

    rows = table.find_all("tr")
    weather_data = []

    for row in rows[1:]:  # Skip the header row
        cols = row.find_all("td")
        cols = [col.text.strip() for col in cols]
        weather_data.append(cols)

    # Define columns for the regular weather data
    columns = [
        "Time",
        "Temperature (°F)",
        "Dew Point",
        "Humidity",
        "Wind",
        "Wind Speed",
        "Wind Gust",
        "Pressure",
        "Precip.",
        "Condition",
    ]

    df = pd.DataFrame(
        weather_data, columns=columns[: len(weather_data[0])]
    )  # Adjust based on actual columns in the table

    driver.quit()

    return df


# Function to loop through multiple dates and save data
def scrape_multiple_days(start_date, end_date):
    # Create a date range
    date_range = pd.date_range(start=start_date, end=end_date)

    all_data = []

    for date in date_range:
        date_str = date.strftime("%Y-%m-%d")
        print(f"Scraping data for {date_str}")
        df = scrape_wunderground_data(date_str)

        if df is not None:
            df["Date"] = date_str  # Add the date to each row
            all_data.append(df)

        # Avoid being blocked
        time.sleep(2)

    # Combine all data into a single DataFrame
    if all_data:
        final_df = pd.concat(all_data, ignore_index=True)
        # Save to CSV
        final_df.to_csv("weather_data.csv", index=False)
        print("Data saved to 'weather_data.csv'")
    else:
        print("No data was scraped.")


# Scrape data from Jan 1, 2023, to Jan 31, 2023 as an example
scrape_multiple_days("2023-01-01", "2023-01-31")

Scraping data for 2023-01-01
Scraping data for 2023-01-02
Scraping data for 2023-01-03
Scraping data for 2023-01-04
Scraping data for 2023-01-05
Scraping data for 2023-01-06
Scraping data for 2023-01-07
Scraping data for 2023-01-08
Scraping data for 2023-01-09
Scraping data for 2023-01-10
Scraping data for 2023-01-11
Scraping data for 2023-01-12
Scraping data for 2023-01-13
Scraping data for 2023-01-14
Scraping data for 2023-01-15
Scraping data for 2023-01-16
Scraping data for 2023-01-17
Scraping data for 2023-01-18
Scraping data for 2023-01-19
Scraping data for 2023-01-20
Scraping data for 2023-01-21
Scraping data for 2023-01-22
Scraping data for 2023-01-23
Scraping data for 2023-01-24
Scraping data for 2023-01-25
Scraping data for 2023-01-26
Scraping data for 2023-01-27
Scraping data for 2023-01-28
Scraping data for 2023-01-29
Scraping data for 2023-01-30
Scraping data for 2023-01-31
Data saved to 'weather_data.csv'


In [ ]:
df_w = pd.read_csv("weather_data.csv")

In [ ]:
df_w.head()

,Time,Temperature (°F),Dew Point,Humidity,Wind,Wind Speed,Wind Gust,Pressure,Precip.,Condition,Date
0,1:54 AM,54 °F,53 °F,97 °%,SSW,10 °mph,0 °mph,29.63 °in,0.0 °in,Cloudy,2023-01-01
1,2:04 AM,54 °F,53 °F,97 °%,SW,12 °mph,0 °mph,29.64 °in,0.0 °in,Cloudy,2023-01-01
2,2:54 AM,54 °F,53 °F,97 °%,WSW,9 °mph,0 °mph,29.62 °in,0.0 °in,Light Rain,2023-01-01
3,3:34 AM,54 °F,53 °F,97 °%,WSW,5 °mph,0 °mph,29.59 °in,0.0 °in,Cloudy,2023-01-01
4,3:54 AM,54 °F,52 °F,93 °%,WSW,12 °mph,0 °mph,29.61 °in,0.0 °in,Cloudy,2023-01-01


In [ ]:
df_weather_scrap = df_w.dropna()

In [ ]:
df_weather_scrap["DateTime"] = df_weather_scrap["Date"] + " " + df_weather_scrap["Time"]

<ipython-input-121-5c53ccc881bf>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_weather_scrap['DateTime'] = df_weather_scrap['Date'] + ' ' + df_weather_scrap['Time']


In [ ]:
# Step 2: Convert the combined 'DateTime' into proper datetime format
# This will handle the AM/PM and convert it to 24-hour time format
df_weather_scrap["DateTime"] = pd.to_datetime(
    df_weather_scrap["DateTime"], format="%Y-%m-%d %I:%M %p"
)

<ipython-input-122-3a8d9bb9908c>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_weather_scrap['DateTime'] = pd.to_datetime(df_weather_scrap['DateTime'], format='%Y-%m-%d %I:%M %p')


In [ ]:
df_weather_scrap.shape

(996, 12)

In [ ]:
df_weather_scrap = df_weather_scrap.drop(columns=["Date", "Time"])

In [ ]:
df_weather_scrap.to_csv("weather_scrap.csv", index=False)

In [ ]:
df_weather_scrap.head()

,Temperature (°F),Dew Point,Humidity,Wind,Wind Speed,Wind Gust,Pressure,Precip.,Condition,DateTime
0,54 °F,53 °F,97 °%,SSW,10 °mph,0 °mph,29.63 °in,0.0 °in,Cloudy,2023-01-01 01:54:00
1,54 °F,53 °F,97 °%,SW,12 °mph,0 °mph,29.64 °in,0.0 °in,Cloudy,2023-01-01 02:04:00
2,54 °F,53 °F,97 °%,WSW,9 °mph,0 °mph,29.62 °in,0.0 °in,Light Rain,2023-01-01 02:54:00
3,54 °F,53 °F,97 °%,WSW,5 °mph,0 °mph,29.59 °in,0.0 °in,Cloudy,2023-01-01 03:34:00
4,54 °F,52 °F,93 °%,WSW,12 °mph,0 °mph,29.61 °in,0.0 °in,Cloudy,2023-01-01 03:54:00


In [ ]:
df_merged.head()

,ride_id,rideable_type,started_at,ended_at,start_station_name,start_station_id,end_station_name,end_station_id,start_lat,start_lng,end_lat,end_lng,member_casual,Closest Weather Station,Weather Station ID,NAME
0,55FDA41F368AD2BB,docked_bike,2023-11-05 15:11:33,2023-11-05 15:31:10,alewife station at russell field,M32033,Harvard Square at Brattle St / Eliot St,M32017,42.396105,-71.139459,42.373329,-71.120986,casual,Central Square,KMACAMBR4,alewife station at russell field
1,EEC4B5509AEE9312,docked_bike,2023-11-07 17:01:55,2023-11-07 17:13:34,main st at austin st,D32020,Gore Street at Lambert Street,M32081,42.375603,-71.064608,42.373080,-71.086342,member,Bellingham Square,KMACHELS7,main st at austin st
2,87158E832C7CB27A,docked_bike,2023-11-09 20:26:49,2023-11-10 07:42:11,purchase st at pearl st,A32026,NaN,NaN,42.354659,-71.053181,NaN,NaN,casual,Union Square,KMASOMER75,purchase st at pearl st
3,844D0BF903C1FCBC,docked_bike,2023-11-30 18:16:44,2023-11-30 18:35:16,seaport square - seaport blvd at northern ave,B32007,Cambridge Crossing at North First Street,M32077,42.351481,-71.044360,42.371141,-71.076198,member,Union Square,KMASOMER75,seaport square - seaport blvd at northern ave
4,FBAE307466B6AC63,docked_bike,2023-11-29 17:09:36,2023-11-29 17:23:58,seaport square - seaport blvd at northern ave,B32007,Cambridge Crossing at North First Street,M32077,42.351481,-71.044360,42.371141,-71.076198,member,Union Square,KMASOMER75,seaport square - seaport blvd at northern ave


In [ ]:
df.shape

(270816, 13)

In [ ]:
df_weather_scrap.shape

(996, 10)

In [ ]:
df_merged["Weather Station ID"].unique()

array(['KMACAMBR4', 'KMACHELS7', 'KMASOMER75', 'KMABOSTO333',
       'KMAROXBU13', 'KMAMALDE19', 'KMABOSTO375', 'KMABOSTO365',
       'KMAEASTB53', 'KMASOUTH405', 'KMABOSTO22', 'KMAMEDFO96',
       'KMACHARL20', 'KMAEASTB63', 'KMAMALDE3', 'KMAREVER50',
       'KMAMALDE16', nan, 'KMAEVERE13'], dtype=object)

In [ ]:
df_merged["Closest Weather Station"].unique()

array(['Central Square', 'Bellingham Square', 'Union Square', 'Kenmore',
       'Roxbury', 'Linden', 'West Concord Street', 'Beacon Hill',
       'Jeffries Point', 'South Boston', 'Boston - South End',
       'South Medford', 'Trenton Street', 'East Boston', 'Malden West',
       'Revere Beach', 'Tea Party', nan, 'Everett'], dtype=object)

In [ ]:
rides_df = df_merged

In [ ]:
rides_df["ended_at"] = pd.to_datetime(rides_df["ended_at"])

# Ensure `DateTime` in weather data is also in datetime format
df_weather_scrap["DateTime"] = pd.to_datetime(df_weather_scrap["DateTime"])

# Step 2: Sort both DataFrames by time (required for merge_asof)
rides_df = rides_df.sort_values("started_at")
df_weather_scrap = df_weather_scrap.sort_values("DateTime")

# Step 3: Perform the asof merge to get the closest weather data for each ride
# Using 'started_at' to find the nearest weather data
merged_df = pd.merge_asof(
    rides_df,
    df_weather_scrap,
    left_on="started_at",
    right_on="DateTime",
    direction="backward",
)

# Step 4: Drop unnecessary columns if needed (like 'DateTime' from weather data)
merged_df = merged_df.drop(columns=["DateTime"])

In [ ]:
merged_df.head()

,ride_id,rideable_type,started_at,ended_at,start_station_name,start_station_id,end_station_name,end_station_id,start_lat,start_lng,...,NAME,Temperature (°F),Dew Point,Humidity,Wind,Wind Speed,Wind Gust,Pressure,Precip.,Condition
0,48AB2DB614847863,docked_bike,2023-11-01 00:00:53,2023-11-01 00:10:56,mit stata center at vassar st / main st,M32005,Kenmore Square,B32010,42.362131,-71.091156,...,mit stata center at vassar st / main st,23 °F,6 °F,48 °%,NNE,8 °mph,0 °mph,30.27 °in,0.0 °in,Partly Cloudy
1,8A16DC870E400CDC,docked_bike,2023-11-01 00:01:13,2023-11-01 00:06:48,harvard square at brattle st / eliot st,M32017,175 N Harvard St,A32019,42.373329,-71.120986,...,harvard square at brattle st / eliot st,23 °F,6 °F,48 °%,NNE,8 °mph,0 °mph,30.27 °in,0.0 °in,Partly Cloudy
2,8E4E81A8A46E5E39,docked_bike,2023-11-01 00:01:23,2023-11-01 00:06:19,landmark center - brookline ave at park dr,B32015,B.U. Central - 725 Comm. Ave.,A32003,42.343691,-71.102353,...,landmark center - brookline ave at park dr,23 °F,6 °F,48 °%,NNE,8 °mph,0 °mph,30.27 °in,0.0 °in,Partly Cloudy
3,8006E6AF3CC4F883,docked_bike,2023-11-01 00:02:39,2023-11-01 00:07:42,tremont st at west st,D32014,Stuart St at Berkeley St,D32049,42.354979,-71.063348,...,tremont st at west st,23 °F,6 °F,48 °%,NNE,8 °mph,0 °mph,30.27 °in,0.0 °in,Partly Cloudy
4,024ED44D1DFF125F,docked_bike,2023-11-01 00:03:11,2023-11-01 00:09:06,colleges of the fenway - fenway at avenue loui...,B32006,Brigham Circle - Francis St at Huntington Ave,B32013,42.340115,-71.100619,...,colleges of the fenway - fenway at avenue loui...,23 °F,6 °F,48 °%,NNE,8 °mph,0 °mph,30.27 °in,0.0 °in,Partly Cloudy


In [ ]:
merged_df = merged_df.dropna()

In [ ]:
merged_df.isnull().sum()

,0
ride_id,0
rideable_type,0
started_at,0
ended_at,0
start_station_name,0
start_station_id,0
end_station_name,0
end_station_id,0
start_lat,0
start_lng,0


In [ ]:
merged_df.shape

(268438, 25)

In [ ]:
rides_df.shape

(270816, 16)